In [2]:
import os, sys, time, resource, re, gc, shutil
from multiprocess import Pool
from functools import partial
from tabulate import tabulate
from urllib.parse import urlparse, parse_qsl

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mongoengine.queryset.visitor import Q
import django
from django.db.models import Count, Sum
sys.path.append('/home/galm/software/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from django.forms.models import model_to_dict

from scoping.models import *
from tmv_app.models import *

In [3]:
run_id = 181
qid = 1281

In [4]:
docs = Doc.objects.filter(query=qid)
wcs = WC.objects.all()
topics = Topic.objects.filter(run_id_id=run_id)

In [5]:
for wc in wcs.iterator():
    print(wc.text)

Oceanography
Ecology
Chemistry, Multidisciplinary
Biodiversity Conservation
Biology
Economics
Geology
Plant Sciences
International Relations
Agriculture, Dairy & Animal Science
Agriculture, Multidisciplinary
Multidisciplinary Sciences
Energy & Fuels
Chemistry, Physical
Environmental Sciences
Parasitology
Physics, Multidisciplinary
Computer Science, Information Systems
Geosciences, Multidisciplinary
Thermodynamics
Meteorology & Atmospheric Sciences
Environmental Studies
Agronomy
Nuclear Science & Technology
Social Sciences, Interdisciplinary
Computer Science, Interdisciplinary Applications
Engineering, Electrical & Electronic
Marine & Freshwater Biology
Geography, Physical
Engineering, Environmental
Biotechnology & Applied Microbiology
Engineering, Chemical
Medicine, General & Internal
Forestry
Engineering, Mechanical
Metallurgy & Metallurgical Engineering
Engineering, Civil
Soil Science
Psychology, Developmental
Engineering, Aerospace
Agricultural Engineering
Food Science & Technology


In [6]:
tvalues = []
for t in topics:
    tv = {"t_id": t.pk, "title":t.title, "score": t.score}
    for wc in wcs.iterator():
        dts = DocTopic.objects.filter(
            run_id=run_id,topic=t,doc__wc=wc
        ).aggregate(
            total = Sum('score')
        )
       
        twcv = {
            "wc_id" : wc.id,
            "wc_text": wc.text,
            "oecd" : wc.oecd,
            "oecd_fos_text" : wc.oecd_fos_text,
            "wc_score" : dts['total']
        }
        twcv.update(tv)
        
        tvalues.append(twcv)
        

In [7]:
df = pd.DataFrame.from_dict(list(tvalues))
df.head()

,oecd,oecd_fos_text,score,t_id,title,wc_id,wc_score,wc_text
0,Natural Sciences,Earth and related environmental sciences,55.284927,20123,"{urban, land, citi}",1,0.033871,Oceanography
1,Natural Sciences,Biological sciences,55.284927,20123,"{urban, land, citi}",236,1.878638,Ecology
2,Natural Sciences,Chemical sciences,55.284927,20123,"{urban, land, citi}",237,0.069853,"Chemistry, Multidisciplinary"
3,Natural Sciences,Biological sciences,55.284927,20123,"{urban, land, citi}",238,0.744505,Biodiversity Conservation
4,Natural Sciences,Biological sciences,55.284927,20123,"{urban, land, citi}",239,0.095993,Biology


In [105]:
pdf = df.pivot(index='title', columns='wc_text', values='wc_score')

In [106]:
pdf

wc_text,Acoustics,Agricultural Economics & Policy,Agricultural Engineering,"Agriculture, Dairy & Animal Science","Agriculture, Multidisciplinary",Agronomy,Allergy,Anatomy & Morphology,Andrology,Anesthesiology,...,Transportation,Transportation Science & Technology,Tropical Medicine,Urban Studies,Urology & Nephrology,Veterinary Sciences,Virology,Water Resources,Women's Studies,Zoology
title,,,,,,,,,,,,,,,,,,,,,
"{activ, school, walk}",0.015315,0.065621,0.030309,NaN,0.057600,NaN,0.071858,NaN,NaN,0.003646,...,0.872240,0.190171,NaN,0.251714,NaN,0.059842,NaN,0.087173,0.026298,0.034406
"{air, pollut, exposur}",0.168633,NaN,0.067016,NaN,0.039782,0.029612,0.076894,NaN,NaN,NaN,...,0.523831,0.286162,NaN,0.156536,NaN,0.075398,NaN,0.038775,NaN,0.009369
"{bicycl, cycl, cyclist}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.802475,0.099332,NaN,0.264471,NaN,NaN,NaN,0.002414,NaN,NaN
"{build, energi, perform}",0.072906,NaN,0.005865,NaN,NaN,NaN,0.026312,NaN,NaN,NaN,...,0.123109,0.047148,NaN,0.069699,0.006406,0.039552,NaN,0.027538,NaN,NaN
"{emiss, carbon, ghg}",0.001324,0.001053,0.025927,NaN,0.027271,NaN,0.002606,NaN,NaN,NaN,...,0.322638,0.117863,NaN,0.136485,0.004801,NaN,NaN,0.012019,NaN,0.004102
"{energi, consumpt, effici}",0.007661,0.004414,0.041496,NaN,0.006445,0.004407,0.006362,NaN,NaN,NaN,...,0.183305,0.165146,NaN,0.120490,0.011225,0.004965,NaN,0.054649,NaN,0.005391
"{green, roof, infrastructur}",NaN,0.003267,0.013683,NaN,0.009502,0.053867,NaN,NaN,NaN,NaN,...,0.095448,0.040006,NaN,0.259977,0.012044,NaN,NaN,0.172374,NaN,NaN
"{heat, thermal, temperatur}",0.025803,NaN,0.037510,NaN,0.012243,NaN,0.028919,NaN,NaN,0.003580,...,0.152659,0.092040,NaN,0.052576,NaN,0.030088,NaN,0.026755,0.002502,0.023791
"{hous, passiv, construct}",0.012085,NaN,NaN,NaN,0.006533,0.001208,0.032386,NaN,NaN,NaN,...,0.086799,0.027896,NaN,0.328908,NaN,NaN,NaN,0.001301,NaN,NaN


In [134]:
import csv

with open('../data/topic_disciplines.csv', 'w', newline='') as csvfile:
    w = csv.writer(csvfile,delimiter='\t')
    

    for name, group in df.groupby('title'):
        w.writerow(['#########################'])
        w.writerow([name])
        g = group.sort_values(by='wc_score', ascending=False).reset_index(drop=True)
        #for gr in g:
        #    w.writerow()
        for i in range(5):
            w.writerow([g['score'][i],g['wc_text'][i],g['wc_score'][i]])
               
        w.writerow([])
    
    

In [124]:
g['score'][0]


54.146646765615202